The vital sign names seems to be mismatching between the crrt patients and controls at UCLA.

Controls have `WEIGHT/SCALE` which is messing with splitting `SBP/DBP`.

In [ ]:
crrt_path = "/home/davina/Private/crrt-data/UCLA"
control_path = "/home/davina/Private/crrt-data/UCLA/controls"
vitals = "Flowsheet_Vitals.txt"

In [ ]:
from pandas import read_csv
from pandas.errors import ParserError

def read_file(file: str):
    try:
        # Try normally reading the csv with pandas, if it fails the formatting is strange
        df = read_csv(file)
    except ParserError as e:
        df = read_csv(file, on_bad_lines="skip")
    except Exception:
        df = read_csv(file, encoding="cp1252")
    return df

crrt_df = read_file(f"{crrt_path}/{vitals}")
ctrl_df = read_file(f"{control_path}/{vitals}")

In [ ]:
crrt_df["VITAL_SIGN_TYPE"].unique()

In [ ]:
ctrl_df["vital_sign_type"].unique()

They all map except for O2 Device, which CRRT people have while controls do not.

In [ ]:
crrt_df[crrt_df["VITAL_SIGN_TYPE"] == "SpO2"]["VITAL_SIGN_VALUE"]

In [ ]:
ctrl_df[ctrl_df["vital_sign_type"] == "PULSE OXIMETRY"]["vital_sign_value"]

In [ ]:
crrt_df[crrt_df["VITAL_SIGN_TYPE"] == "O2 Device"]["VITAL_SIGN_VALUE"]

# time difference of different explode procedures

In [ ]:
sample = crrt_df.sample(10000)
sample["VITAL_SIGN_TYPE"].replace(
        {"BP": "SBP/DBP", "BLOOD PRESSURE": "SBP/DBP"}, inplace=True
    )
explode_cols = ["VITAL_SIGN_VALUE", "VITAL_SIGN_TYPE"]

In [ ]:
%%timeit -n 100

(
    sample.set_index(list(sample.columns.difference(explode_cols)))
        .apply(lambda col: col.str.split("/").explode())
        .reset_index()
        .reindex(sample.columns, axis=1)
)

In [ ]:
%%timeit  -n 100
b = (
	sample.apply(
		lambda col: col.str.split("/") 
		if col.name in explode_cols else col
	).explode(explode_cols)
)

In [ ]:
a = (
    sample.set_index(list(sample.columns.difference(explode_cols)))
        .apply(lambda col: col.str.split("/").explode())
        .reset_index()
        .reindex(sample.columns, axis=1)
)
b = (
	sample.apply(
		lambda col: col.str.split("/") 
		if col.name in explode_cols else col
	).explode(explode_cols)
    .reset_index(drop=True)  # reset index to be fair comparison
)
a.equals(b)